In [ ]:
# Import modules
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

In [165]:
# Test: getting description of google provider
url_provider_google = "https://registry.terraform.io/v2/providers/hashicorp/google"
response_provider_google = requests.get(url_provider_google)
data_provider_google = json.loads(response_provider_google.text)
print(data_provider_google)

{'data': {'type': 'providers', 'id': '354', 'attributes': {'alias': 'google', 'description': 'Lifecycle management of GCP resources, including Compute Engine, Cloud Storage, Cloud SDK, Cloud SQL, GKE, BigQuery, Cloud Functions and more. This provider is collaboratively maintained by the Google Terraform Team at Google and the Terraform team at HashiCorp', 'downloads': 155056732, 'featured': True, 'full-name': 'hashicorp/google', 'logo-url': '/images/providers/google-cloud.svg', 'name': 'google', 'namespace': 'hashicorp', 'owner-name': 'hashicorp', 'robots-noindex': False, 'source': 'https://github.com/hashicorp/terraform-provider-google', 'tier': 'official', 'unlisted': False, 'warning': ''}, 'links': {'self': '/v2/providers/354'}}}


In [170]:
# Getting the list of all modules of v2 API
url = "https://registry.terraform.io/v2/modules"
response = requests.get(url)
data = json.loads(response.text)
print(len(data['data']))
print(data)
#df = pd.DataFrame(data['meta'])
#df.plot(kind='bar')
#plt.show()

15
{'data': [{'type': 'modules', 'id': '23', 'attributes': {'downloads': 23550103, 'full-name': 'terraform-aws-modules/vpc/aws', 'name': 'vpc', 'namespace': 'terraform-aws-modules', 'owner-name': '', 'provider-logo-url': '/images/providers/aws.png', 'provider-name': 'aws', 'source': 'https://github.com/terraform-aws-modules/terraform-aws-vpc', 'verified': True}, 'links': {'self': '/v2/modules/23'}}, {'type': 'modules', 'id': '179', 'attributes': {'downloads': 16974738, 'full-name': 'cloudposse/label/null', 'name': 'label', 'namespace': 'cloudposse', 'owner-name': '', 'provider-logo-url': '/images/providers/generic.svg?2', 'provider-name': 'null', 'source': 'https://github.com/cloudposse/terraform-null-label', 'verified': False}, 'links': {'self': '/v2/modules/179'}}, {'type': 'modules', 'id': '451', 'attributes': {'downloads': 14388064, 'full-name': 'terraform-aws-modules/security-group/aws', 'name': 'security-group', 'namespace': 'terraform-aws-modules', 'owner-name': '', 'provider-lo

In [ ]:
# Next iterate over the page of each API call to store all available modules in a list
# Then use this data to filter for provider like aws, google, azurerm to get the count of modules for each provider
# Then plot the count of modules for each provider